In [1]:
import wandb
import pandas as pd

api = wandb.Api(overrides={'base-url': "https://api.wandb.ai"})
runs = api.runs("point-process/llmforecast")

In [2]:
print(runs)

<Runs point-process/llmforecast>


In [20]:
def create_summary_table(runs, row_name, multi_column_name, column_name, metric):
    # Initialize a dictionary for storing the lowest metric values
    lowest_metrics = {}
    run_mapping = {}

    # Find the run with the lowest metric for each combination
    for run in runs:
        try:
            name = run.config[row_name]
            dataset = run.config[multi_column_name]
            window_size = run.config[column_name] if column_name else None
            metric_value = float(run.summaryMetrics[metric])
        except KeyError:
            continue

        key = (name, dataset) if column_name is None else (name, dataset, window_size)

        if key not in lowest_metrics:
            lowest_metrics[key] = metric_value
            run_mapping[key] = run

    # Prepare the data structure for DataFrame creation
    index = sorted(set(key[0] for key in run_mapping.keys()))
    if column_name:
        datasets = sorted(set(key[1] for key in run_mapping.keys()))
        window_sizes = sorted(set(key[2] for key in run_mapping.keys()))
        multi_columns = pd.MultiIndex.from_product([datasets, window_sizes], names=[multi_column_name, column_name])
    else:
        multi_columns = sorted(set(key[1] for key in run_mapping.keys()))

    # Create the DataFrame
    results_df = pd.DataFrame(index=index, columns=multi_columns)

    # Populate the DataFrame
    for key, run in run_mapping.items():
        if column_name:
            results_df.at[key[0], (key[1], key[2])] = run.summaryMetrics[metric]
        else:
            results_df.at[key[0], key[1]] = run.summaryMetrics[metric]

    # Drop rows and columns with all NaN values
    results_df.dropna(axis=0, how='all', inplace=True)
    results_df.dropna(axis=1, how='all', inplace=True)

    return results_df

In [21]:
results_df = create_summary_table(runs, 'name', 'dataset', 'window_size', 'rmse')
results_df

dataset                    Climate                                   \
window_size                     7          9          14         30   
Llama-2-7b-chat-hf        0.179627   0.151682        NaN        NaN   
Mistral-7B-Instruct-v0.1  0.183741   0.264709        NaN        NaN   
arima                     0.280252   0.200725   0.138294   0.137508   
avg                        0.11711   0.115335   0.118583   0.139972   
ets                       0.214807    0.20287   0.168499   0.164285   
gemma-7b-it               0.135962   0.257995        NaN        NaN   
last                        0.1405   0.141434   0.142096   0.153495   
llama                     45.96401  40.331071  37.557077  33.482239   
llmtime                   0.119366   0.124417        NaN        NaN   
lltime                     0.73183   7.717368        NaN        NaN   
lltime-mistral7b-v0       0.743267   0.339105        NaN        NaN   
m2zeroshot                0.133946   0.138263   0.133167   0.166866   
nlinear_darts             0.115071   0.113953   0.117419   0.128813   
t5-test                   0.174831   0.183719   0.186428   0.188907   
t5-test-only-ts           0.167231   0.173385   0.186516   0.212582   
t5-test-xval              0.333891   0.449173        NaN        NaN   
tide_darts                0.118599   0.122203   0.134031   0.138649   

dataset                         Mimic                                     \
window_size                        5            7           9         14   
Llama-2-7b-chat-hf           0.154039     0.186128    0.183625       NaN   
Mistral-7B-Instruct-v0.1      0.15644     0.190862    0.262242       NaN   
arima                        0.241965     0.274563    0.216994  0.198986   
avg                          0.140004     0.147728    0.154714  0.174503   
ets                          0.209988     0.207661     0.20871  0.212283   
gemma-7b-it                  0.166181     0.181875    0.269328       NaN   
last                         0.151323     0.158277    0.164154  0.177461   
llama                     2134.282728  1373.864837  868.926823       NaN   
llmtime                      0.127848     0.144255    0.151237       NaN   
lltime                       0.203834     0.191714    0.205618       NaN   
lltime-mistral7b-v0          0.273616     0.363217    0.333222       NaN   
m2zeroshot                   0.118424     0.114694    0.126514  0.131506   
nlinear_darts                0.135405     0.143497     0.14952  0.165366   
t5-test                      0.184758     0.192787    0.199776  0.220086   
t5-test-only-ts              0.182591     0.189422    0.203977  0.210137   
t5-test-xval                      NaN     0.388963         NaN       NaN   
tide_darts                   0.135895     0.144814    0.155226    0.1718   

dataset                       Yelp                                
window_size                     4         8         12        24  
Llama-2-7b-chat-hf         0.75105  0.771902  0.888398       NaN  
Mistral-7B-Instruct-v0.1  0.736506  0.777644  0.881798       NaN  
arima                     1.191419    1.3808  0.878099  0.800031  
avg                       0.704788  0.693359  0.701275  0.710008  
ets                       1.367694  1.334058  1.020702  1.160937  
gemma-7b-it               0.877593  1.419001  1.490367       NaN  
last                        0.8442  0.866989  0.883912  0.914723  
llama                          NaN       NaN       NaN       NaN  
llmtime                    0.58706  0.663245  0.693942       NaN  
lltime                    1.439553  1.953058  1.105995  0.985528  
lltime-mistral7b-v0       1.167544  1.593075       NaN       NaN  
m2zeroshot                0.799638  0.793335  0.846942  7.331404  
nlinear_darts             0.679195   0.68474  0.690688   0.70554  
t5-test                   0.847052  0.870443  0.845917  0.843526  
t5-test-only-ts           0.889274  0.878499  0.884224  0.877887  
t5-test-xval              1.668003  1.753296  2.050931       NaN  

In [22]:
def postprocess_df(df, rows_to_remove, columns_to_remove, precision):
    # Define a lambda to round and format the float to the desired precision
    format_float = lambda x: f"{x:.{precision}f}" if isinstance(x, float) else x
    
    # Remove specified rows and columns
    df = df.drop(index=rows_to_remove)
    df = df.drop(columns=columns_to_remove, axis=1)
    
    # Convert all values to the specified precision
    df = df.applymap(lambda x: format_float(round(float(x), precision)) if pd.notnull(x) and isinstance(x, (int, float)) else x)
    
    # Define a function to apply bold styling to the minimum value in each column
    def highlight_min(s):
        s = pd.to_numeric(s, errors='coerce')
        is_min = s == s.min()
        return ['font-weight: bold' if v else '' for v in is_min]
    
    # Apply the styling with the Styler object
    styled_df = df.style.apply(highlight_min, axis=0)
    
    return styled_df, df


# Example usage:
rows_to_remove = ['llama', 'm2zeroshot'] # replace with your actual row values to remove
columns_to_remove = [('Yelp', 24), ('Mimic', 14), ('Climate', 30), ('Climate', 14)] # replace with your actual column values to remove
precision = 3

processed_df, df = postprocess_df(results_df, rows_to_remove, columns_to_remove, precision)
processed_df

/tmp/ipykernel_1075699/2864370779.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: format_float(round(float(x), precision)) if pd.notnull(x) and isinstance(x, (int, float)) else x)


In [11]:
df.to_csv('results.csv')